# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f933ef572e0>
├── 'a' --> tensor([[-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305]])
└── 'x' --> <FastTreeValue 0x7f933ef570a0>
    └── 'c' --> tensor([[ 0.0293,  0.3646,  1.4814,  0.4672],
                        [-0.7520, -2.1412, -0.5661, -0.6556],
                        [ 0.3202,  0.3797,  0.7924,  1.1524]])

In [4]:
t.a

tensor([[-1.2273, -0.5245, -1.8725],
        [-0.4215, -1.6120,  0.8305]])

In [5]:
%timeit t.a

73.4 ns ± 0.0396 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f933ef572e0>
├── 'a' --> tensor([[0.8709, 0.0956, 0.9968],
│                   [1.1956, 0.5418, 0.3427]])
└── 'x' --> <FastTreeValue 0x7f933ef570a0>
    └── 'c' --> tensor([[ 0.0293,  0.3646,  1.4814,  0.4672],
                        [-0.7520, -2.1412, -0.5661, -0.6556],
                        [ 0.3202,  0.3797,  0.7924,  1.1524]])

In [7]:
%timeit t.a = new_value

75.9 ns ± 0.052 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-1.2273, -0.5245, -1.8725],
               [-0.4215, -1.6120,  0.8305]]),
    x: Batch(
           c: tensor([[ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524]]),
       ),
)

In [10]:
b.a

tensor([[-1.2273, -0.5245, -1.8725],
        [-0.4215, -1.6120,  0.8305]])

In [11]:
%timeit b.a

69 ns ± 0.0348 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.9159,  0.3611,  0.3908],
               [ 0.1084, -0.1231,  1.2568]]),
    x: Batch(
           c: tensor([[ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524]]),
       ),
)

In [13]:
%timeit b.a = new_value

550 ns ± 1.19 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

879 ns ± 8.41 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 9.61 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

147 µs ± 995 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

150 µs ± 1.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f926f1f4070>
├── 'a' --> tensor([[[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]],
│           
│                   [[-1.2273, -0.5245, -1.8725],
│                    [-0.4215, -1.6120,  0.8305]]])
└── 'x' --> <FastTreeValue 0x7f926f0fad60>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33.1 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f933ef57610>
├── 'a' --> tensor([[-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305],
│                   [-1.2273, -0.5245, -1.8725],
│                   [-0.4215, -1.6120,  0.8305]])
└── 'x' --> <FastTreeValue 0x7f926f0faee0>
    └── 'c' --> tensor([[ 0.0293,  0.3646,  1.4814,  0.4672],
                        [-0.7520, -2.1412, -0.5661, -0.6556],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 19.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

67.7 µs ± 31.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.0293,  0.3646,  1.4814,  0.4672],
                       [-0.7520, -2.1412, -0.5661, -0.6556],
                       [ 0.3202,  0.3797,  0.7924,  1.1524]],
              
                      [[ 0.0293,  0.3646,  1.4814,  0.4672],
                       [-0.7520, -2.1412, -0.5661, -0.6556],
                       [ 0.3202,  0.3797,  0.7924,  1.1524]],
              
                      [[ 0.0293,  0.3646,  1.4814,  0.4672],
                       [-0.7520, -2.1412, -0.5661, -0.6556],
                       [ 0.3202,  0.3797,  0.7924,  1.1524]],
              
                      [[ 0.0293,  0.3646,  1.4814,  0.4672],
                       [-0.7520, -2.1412, -0.5661, -0.6556],
                       [ 0.3202,  0.3797,  0.7924,  1.1524]],
              
                      [[ 0.0293,  0.3646,  1.4814,  0.4672],
                       [-0.7520, -2.1412, -0.5661, -0.6556],
                       [ 0.3202,  0.3797,  0.7924,  1.1524]],

In [26]:
%timeit Batch.stack(batches)

84.2 µs ± 345 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524],
                      [ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524],
                      [ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524],
                      [ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524],
                      [ 0.0293,  0.3646,  1.4814,  0.4672],
                      [-0.7520, -2.1412, -0.5661, -0.6556],
                      [ 0.3202,  0.3797,  0.7924,  1.1524],
                      [ 0.0293,  0.3646,  1.4814,  0.4672],
                   

In [28]:
%timeit Batch.cat(batches)

153 µs ± 517 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

358 µs ± 6.1 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
